In [1]:
import pandas as pd
import numpy
from tqdm.notebook import trange, tqdm

In [2]:
data_file = 'data/wikitext-2/train.txt'

In [59]:
window_size = 7

def get_tokens(data_file):
    with open(data_file) as d:
        for line in d:
            line = line.strip()
            if len(line) > 0:
                l = []
                for split_at in range(0, len(line), window_size):
                    l.append(line[split_at:split_at+window_size])
                yield l

                
def get_charmap(data_file):
    charmap = {}
    with open(data_file) as d:
        for line in d:
            chars = line.strip()
            for char in chars:
                if char not in charmap:
                    charmap[char] = len(charmap)
    return charmap

def get_unigrams(data_file):
    charmap = {}
    with open(data_file) as d:
        for line in d:
            chars = line.strip().split(' ')
            for char in chars:
                if char not in charmap:
                    charmap[char] = 0
                charmap[char] += 1
    return charmap


In [4]:
charmap = get_charmap(data_file)

In [5]:
charmap_len = len(charmap)

In [6]:
from ipywidgets import IntProgress
IntProgress(10,max=100)

IntProgress(value=10)

In [7]:
%%time
from collections import Counter
candidates = {}

context_window_size = 3

for line in tqdm(get_tokens(data_file)):
    #print(line)
    for t in range(len(line)):
        tok = line[t]
        context_left_size = max(t - context_window_size, 0)
        context_right_size = min(t + 1 + context_window_size, len(line))
        context_left = ''.join(line[context_left_size:t])
        context_right = ''.join(line[t + 1: context_right_size])
        #print(context_left, tok, context_right)
        for i in range(len(tok)):
            for j in range(i, len(tok)):
                right_context = tok[j+1:] + context_right
                left_context =  context_left + tok[:i]
                total_context = left_context + right_context
                candidate = tok[i:j+1]
                if candidate not in candidates:
                    candidates[candidate] = numpy.zeros(charmap_len)
                for c in total_context:
                    candidates[candidate][charmap[c]] += 1
                #candidates[candidate]
        #break
        #print()



CPU times: user 21min 54s, sys: 8.16 s, total: 22min 2s
Wall time: 42min 4s


In [11]:
len(candidates)

1326020

In [54]:
word = 'cats'
word_vec = candidates[word]
#chars = [candidates[p] for p in word]
chars = [candidates[p] for p in ['cat', 's ']]

sum_chars = sum(chars) 
#sum_chars = sum_chars/numpy.linalg.norm(sum_chars)

In [55]:
from scipy.spatial.distance import cosine

In [56]:
cosine(word_vec, sum_chars)

0.019143108100468487

In [65]:
cosine(candidates['r'], candidates['role'])

0.004580065983352255

In [60]:
unigrams = get_unigrams(data_file)

In [66]:
unigrams['cat']

32